### LIBRARIES

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

### DATA LOADING AND VISUALISATION

In [2]:
df = pd.read_csv("Balanced_dataset.csv")
print(df.head())  # Show first 5 rows
print(df.info())  # Get data types and non-null values

                                                 url    type scheme  \
0    https://ethnicelebs.com/eddie-anderson-comedian  benign  https   
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign  https   
2  https://terezowens.com/golfer-greg-norman-tryi...  benign  https   
3                        https://americannortel.com/  benign  https   
4                               https://welding.org/  benign  https   

               domain subdomain top_level_domain  \
0     ethnicelebs.com      none              com   
1    closinglogos.com      none              com   
2      terezowens.com      none              com   
3  americannortel.com      none              com   
4         welding.org      none              org   

                                                path  path_length  \
0                           /eddie-anderson-comedian           24   
1  /page/Sony+Pictures+Home+Entertainment+Warning...           53   
2  /golfer-greg-norman-trying-to-unload-55-millio... 

In [3]:
df

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59600,http://archives.li.man.ac.uk/ead/search/?opera...,spam,http,archives.li.man.ac.uk,archives,uk,/ead/search/,12,2,6,6,0,none,0,0,0,0,0
59601,http://amber.ch.ic.ac.uk/archive/all/10253.html,spam,http,amber.ch.ic.ac.uk,amber,uk,/archive/all/10253.html,23,3,0,0,0,html,0,0,0,0,0
59602,http://archive.thisischeshire.co.uk/2000/1/28/...,spam,http,archive.thisischeshire.co.uk,archive,uk,/2000/1/28/225238.html,22,4,0,0,0,html,0,0,0,0,0
59603,http://ads.guardian.co.uk/click.ng/Params.rich...,spam,http,ads.guardian.co.uk,ads,uk,/click.ng/Params.richmedia=yes&location=middle...,128,2,0,0,0,richmedia=yes&location=middle&spacedesc=06&sit...,0,1,0,0,0


In [4]:
df.columns

Index(['url', 'type', 'scheme', 'domain', 'subdomain', 'top_level_domain',
       'path', 'path_length', 'num_path_segments', 'query_params',
       'num_query_params', 'has_https', 'file_extension', 'has_fragment',
       'has_special_chars_in_path', 'has_port', 'port_number',
       'is_ip_address'],
      dtype='object')

In [5]:
df['type'].value_counts()

type
benign        11921
defacement    11921
phishing      11921
malware       11921
spam          11921
Name: count, dtype: int64

In [7]:
df[df.columns][:5]

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0


### USING TEXTUAL FEATURES ONLY

In [8]:
df2 = pd.read_csv("Balanced_dataset.csv")
df2 = df2.fillna("")
df2["text"] = df2["url"].astype(str) + " " + df2["scheme"].astype(str) + " " + df2["domain"].astype(str) + " " + \
              df2["subdomain"].astype(str) + " " + df2["top_level_domain"].astype(str) + " " + \
              df2["path"].astype(str) + " " + df2["file_extension"].astype(str)
assert df2["text"].apply(lambda x: isinstance(x, str)).all(), "Error: Some values are not strings!"
print(df2[["text", "type"]].head())

                                                text    type
0  https://ethnicelebs.com/eddie-anderson-comedia...  benign
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign
2  https://terezowens.com/golfer-greg-norman-tryi...  benign
3  https://americannortel.com/ https americannort...  benign
4  https://welding.org/ https welding.org none or...  benign


In [9]:
df2[["text", "type"]]

,text,type
0,https://ethnicelebs.com/eddie-anderson-comedia...,benign
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign
2,https://terezowens.com/golfer-greg-norman-tryi...,benign
3,https://americannortel.com/ https americannort...,benign
4,https://welding.org/ https welding.org none or...,benign
...,...,...
59600,http://archives.li.man.ac.uk/ead/search/?opera...,spam
59601,http://amber.ch.ic.ac.uk/archive/all/10253.htm...,spam
59602,http://archive.thisischeshire.co.uk/2000/1/28/...,spam
59603,http://ads.guardian.co.uk/click.ng/Params.rich...,spam


In [10]:
import torch

# Check if GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")


Using device: cuda


### UTILIZING ALBERTA-BASE-V2 Pre-Trained

In [11]:
model_name = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(text_list):
    return tokenizer(text_list, padding="max_length", truncation=True, return_tensors="pt")
tokenized_data = tokenize_function(df2["text"].tolist())
print(tokenized_data.keys())  # Check available keys (input_ids, attention_mask, etc.)

C:\Users\moawi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


### DATA LOADER CLASS

In [12]:
class URLDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
label_mapping = {"benign": 0, "defacement": 1, "phishing": 2, "malware": 3, "spam": 4}
df2["label"] = df2["type"].map(label_mapping)
dataset = URLDataset(tokenized_data, df2["label"].tolist())

### TRAINING AND TESTING SPLIT

In [13]:

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
print(f"Training Samples: {len(train_dataset)}, Validation Samples: {len(test_dataset)}")

Training Samples: 47684, Validation Samples: 11921


In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=5  # Change '2' based on the number of classes in your dataset
).to(device)

C:\Users\moawi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### MODEL TRAINER

In [15]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


# Define training arguments
training_args = TrainingArguments(
    output_dir="./info sec a2/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./info sec a2/logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False,  # Don't use fp16 on Ampere GPUs, use bf16 instead
    bf16=True,   # Enable bf16 mixed precision for faster training
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



### MODEL TRAINING

In [16]:
# Train the model
trainer.train()

C:\Users\moawi\AppData\Local\Temp\ipykernel_29180\268079057.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.205300,0.272696,0.928781,0.930634,0.928781,0.928749
2,0.149900,0.191909,0.950759,0.951519,0.950759,0.950791
3,0.125100,0.172593,0.958309,0.958867,0.958309,0.958470


C:\Users\moawi\AppData\Local\Temp\ipykernel_29180\268079057.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\moawi\AppData\Local\Temp\ipykernel_29180\268079057.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\moawi\AppData\Local\Temp\ipykernel_29180\268079057.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in s

TrainOutput(global_step=8943, training_loss=0.23970924628256793, metrics={'train_runtime': 48800.374, 'train_samples_per_second': 2.931, 'train_steps_per_second': 0.183, 'total_flos': 3419685077372928.0, 'train_loss': 0.23970924628256793, 'epoch': 3.0})

### MODEL EVALUATION

In [17]:
metrics = trainer.evaluate()
print(metrics)


C:\Users\moawi\AppData\Local\Temp\ipykernel_29180\268079057.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.17259261012077332, 'eval_accuracy': 0.9583088667058133, 'eval_precision': 0.9588674649800065, 'eval_recall': 0.9583088667058133, 'eval_f1': 0.9584696029973567, 'eval_runtime': 679.4164, 'eval_samples_per_second': 17.546, 'eval_steps_per_second': 1.098, 'epoch': 3.0}
